In [1]:
%pip install --upgrade --quiet google-cloud-aiplatform

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 64.0 MB/s eta 0:00:00


In [2]:
from inspect import cleandoc
from IPython.display import display, Markdown

import vertexai
from vertexai.generative_models import GenerativeModel, GenerationConfig

In [3]:
PROJECT_ID = "qwiklabs-gcp-03-34639a9a7f8b"
LOCATION = "us-central1"
import vertexai
vertexai.init(project=PROJECT_ID, location=LOCATION)

## **Load a generative model**

In [4]:
model = GenerativeModel("gemini-2.0-flash-001")

/usr/local/lib/python3.11/dist-packages/vertexai/generative_models/_generative_models.py:433: UserWarning: This feature is deprecated as of June 24, 2025 and will be removed on June 24, 2026. For details, see https://cloud.google.com/vertex-ai/generative-ai/docs/deprecations/genai-vertexai-sdk.
  warning_logs.show_deprecation_warning()


## **Define the output format & specify constraints**

In [5]:
transcript = """
    Speaker 1 (Customer): Hi, can I get a cheeseburger and large fries, please?
    Speaker 2 (Restaurant employee): Coming right up! Anything else you'd like to add to your order?
    Speaker 1: Hmmm, maybe a small orange juice. And could I get the fries with ketchup on the side?
    Speaker 2: No problem, one cheeseburger, one large fries with ketchup on the side, and a small
    orange juice. That'll be $5.87. Drive through to the next window please.
"""

**Following prompt attempts to understand a customer's order from a conversation in JSON format, but in an unspecific way.**

In [11]:
response = model.generate_content(f"""
    Extract the transcript to JSON.

    {transcript}
""")

display(Markdown(response.text))

```json
[
  {
    "speaker": "Customer",
    "utterance": "Hi, can I get a cheeseburger and large fries, please?"
  },
  {
    "speaker": "Restaurant employee",
    "utterance": "Coming right up! Anything else you'd like to add to your order?"
  },
  {
    "speaker": "Customer",
    "utterance": "Hmmm, maybe a small orange juice. And could I get the fries with ketchup on the side?"
  },
  {
    "speaker": "Restaurant employee",
    "utterance": "No problem, one cheeseburger, one large fries with ketchup on the side, and a small orange juice. That'll be $5.87. Drive through to the next window please."
  }
]
```

**Prompt with more specific instructions detailing the exact structure expected in the output. Notice how the JSON output now reflects the key pieces of information relevant to understanding the user's order.**

In [12]:
response = model.generate_content(f"""
    <INSTRUCTIONS>
    - Extract the ordered items into JSON.
    - Separate drinks from food.
    - Include a quantity for each item and a size if specified.
    </INSTRUCTIONS>

    <TRANSCRIPT>
    {transcript}
    </TRANSCRIPT>
""")

display(Markdown(response.text))

```json
{
  "food_items": [
    {
      "item": "cheeseburger",
      "quantity": 1
    },
    {
      "item": "fries",
      "quantity": 1,
      "size": "large",
      "notes": "ketchup on the side"
    }
  ],
  "drink_items": [
    {
      "item": "orange juice",
      "quantity": 1,
      "size": "small"
    }
  ]
}
```

## **Assign a persona or role**

In [8]:
chat = model.start_chat()

**Response without a persona specified**

In [13]:
response = chat.send_message(
    """
    Provide a brief guide to caring for the houseplant monstera deliciosa?
    """
)

display(Markdown(response.text))

## Monstera Deliciosa: Quick Care Guide

Here's the need-to-know for a happy Monstera Deliciosa (Swiss Cheese Plant):

**Light:** Bright, indirect light is best. Avoid direct sun.

**Water:** Water when the top 1-2 inches of soil are dry. Don't overwater!

**Humidity:** Loves humidity. Mist, use a humidifier, or pebble tray.

**Soil:** Well-draining potting mix is essential.

**Support:** Provide a moss pole or trellis for climbing and bigger leaves.

**Fertilizer:** Feed during growing season (spring/summer) every 2-4 weeks.

**Key Takeaways:** Bright light, careful watering, and some humidity will keep your Monstera thriving!


**Response with a role specified**

In [14]:
new_chat = model.start_chat()

response = new_chat.send_message(
    """
    You are a houseplant monstera deliciosa. Help the person who
    is taking care of you to understand your needs.
    """
)

display(Markdown(response.text))

Okay, okay, hello there! I'm Monty, your friendly neighborhood Monstera Deliciosa, and I just wanted to have a little chat about how you can keep me thriving!  Think of this as a little "Monstera Maintenance 101," alright?  Let's dive in:

**1. Light is My Love Language (But Not *Direct* Sunlight!)**

*   **What I crave:** Bright, indirect light. Imagine dappled sunlight filtering through a forest canopy. That's my jam!
*   **What I DON'T want:** Scorching direct sunlight.  It'll give me a sunburn faster than you can say "Swiss cheese plant!" (Seriously, my leaves will get yellow and crispy – not cute.)
*   **Where to put me:** Near an east-facing window is great, or a few feet back from a south or west-facing window.  If the light is too strong, put up a sheer curtain to diffuse it.
*   **Signs of being unhappy:**
    *   **Pale green or yellow leaves:** Probably too much direct sunlight.
    *   **Small leaves, long leggy growth, no fenestrations (splits):**  Not enough light!  I'm struggling to photosynthesize.

**2. Water: I'm Thirsty, But Not *Too* Thirsty!**

*   **How to water me:**  Water me thoroughly when the top 1-2 inches of soil feel dry to the touch.  Stick your finger in there – don't be shy!
*   **Drainage is KEY!**  Make sure my pot has drainage holes. I *hate* sitting in soggy soil; it leads to root rot, and that's a death sentence!
*   **Don't water on a schedule!**  The amount of light, humidity, and temperature will all affect how quickly my soil dries out.  Always check the soil moisture before watering.
*   **Signs of being unhappy:**
    *   **Yellowing leaves:** Could be overwatering OR underwatering! Check the soil. If it's soggy, it's overwatering. If it's bone dry, it's underwatering.
    *   **Drooping leaves:**  Again, could be either!  Overwatering is the more common culprit, though.
    *   **Brown, crispy tips:** Usually underwatering or low humidity.

**3. Humidity: I Like to Feel the Mist!**

*   **I'm a tropical plant, baby!**  I thrive in higher humidity.
*   **How to boost humidity:**
    *   **Mist me regularly:**  A light spritz with a spray bottle will make me happy.
    *   **Use a humidifier:** If your home is dry, a humidifier is a fantastic investment.
    *   **Group me with other plants:**  Plants help create their own little microclimate.
    *   **Pebble tray:** Place my pot on a tray filled with pebbles and water (making sure the bottom of the pot doesn't sit *in* the water!).  As the water evaporates, it increases the humidity around me.
*   **Signs of being unhappy:**
    *   **Brown, crispy leaf edges:** Lack of humidity is often the culprit.

**4. Soil & Fertilizer: Nutrients are My Snacks!**

*   **Well-draining soil is essential!** A mix of potting soil, perlite, and orchid bark is ideal.
*   **Fertilize me during the growing season (spring and summer):** Use a balanced liquid fertilizer diluted to half strength every 2-4 weeks.
*   **Lay off the fertilizer in the fall and winter:** I'm not actively growing then, so I don't need as much food.

**5. Support: Help Me Climb, Baby!**

*   **I'm a climber by nature!**  Eventually, I'll need a support to grow on.
*   **What to use:**  A moss pole, a trellis, or even just a sturdy stake will work.
*   **Why it's important:**  Giving me support will encourage me to grow larger leaves and develop those signature fenestrations (splits)!

**6.  Pruning: A Little Trim is Okay!**

*   **You can prune me to control my size or remove damaged leaves.**  Just use clean, sharp scissors or pruning shears.
*   **Don't be afraid to chop!**  I'm pretty resilient.
*   **You can even propagate cuttings!**  Look up "Monstera Deliciosa propagation" for instructions.

**7. Watch Out for Pests!**

*   **Common pests:**  Spider mites, mealybugs, and scale.
*   **Check me regularly:**  Inspect my leaves (especially the undersides) for any signs of pests.
*   **Treatment:**  If you find pests, isolate me from other plants and treat me with insecticidal soap or neem oil.

**Most Importantly: Pay Attention to Me!**

I can't actually talk (though wouldn't that be awesome?), so I rely on you to observe my leaves and overall appearance to figure out what I need. A little observation goes a long way!  

If you follow these tips, I promise to reward you with lush, beautiful foliage and those amazing iconic fenestrations! Good luck, and happy growing!  Let me know if you have any more questions – I'm always here to help (well, as much as a plant can help!).  Now, if you'll excuse me, I think I feel a little thirsty...


## **Include examples**

A prompt that includes no examples is called a **zero-shot prompt**. One with a single example is a **one-shot prompt**. And a few examples would make it a **few-shot prompt**.

In [17]:
question = """
We offer software consulting services. Read a potential
customer's message and rank them on a scale of 1 to 3
based on whether they seem likely to hire us for our
developer services within the next month. Return the likelihood
rating labeled as "Likelihood: SCORE".
Do not include any Markdown styling.

1 means they are not likely to hire.
2 means they might hire, but they are not likely ready to do
so right away.
3 means they are looking to start a project soon.

Example Message: Hey there I had an idea for an app,
and I have no idea what it would cost to build it.
Can you give me a rough ballpark?
Likelihood: 1

Example Message: My department has been using a vendor for
our development, and we are interested in exploring other
options. Do you have time for a discussion around your
services?
Likelihood: 2

Example Message: I have mockups drawn for an app and a budget
allocated. We are interested in moving forward to have a
proof of concept built within 2 months, with plans to develop
it further in the following quarter.
Likelihood: 3

Customer Message: Our department needs a custom gen AI solution.
We have a budget to explore our idea. Do you have capacity
to get started on something soon?
 """

response = model.generate_content(question)

display(Markdown(response.text))

Likelihood: 3


## **Experiment with parameter values**

The temperature and top_p parameters which lead to variety in responses are set to low values, so the output should be the same, or very close to the same, each time.

In [18]:
response = model.generate_content(
    """
    Tell me a joke about frogs.
    """,
    generation_config={"top_p": .05,
                       "temperature": 0.05}
)

display(Markdown(response.text))

Why did the frog call his insurance company?

Because he got toad!


In [19]:
response = model.generate_content(
    """
    Tell me a joke about frogs.
    """,
    generation_config={"top_p": .05,
                       "temperature": 0.05}
)

display(Markdown(response.text))

Why did the frog call his insurance company?

Because he got toad!


The higher temperature and top_p parameter values now lead to more varied results. Some of the results, however, may be a little too random and not make very much sense. If you want variety in your responses, you'll need to experiment with parameters to determine the right balance of creativity and reliability.

In [20]:
response = model.generate_content(
    """
    Tell me a joke about frogs.
    """,
    generation_config={"top_p": .05,
                       "temperature":1}
)

display(Markdown(response.text))

Why did the frog call a plumber?

Because he had a croak in his throat!


In [26]:
response = model.generate_content(
    """
    Tell me a joke about frogs.
    """,
    generation_config={"top_p": .98,
                       "temperature":1}
)

display(Markdown(response.text))

Why did the frog call his insurance company?

Because he got toad!


***One parameter you will want to set consistently is a temperature of 0 when you are working on mathematical, logical, precise question-answering, or other problems where you want only the most correct answer.***




## fallback responses
A fallback response is a response the model should use when a user input would take the conversation out of your intended scope. It can provide the user a polite response that directs them back to the intended topic.

In [27]:
response = model.generate_content(
    """
    Instructions: Answer questions about pottery.
    If a user asks about something else, reply with:
    Sorry, I only talk about pottery!

    User Query: How high can a horse jump?
    """
)
display(Markdown(response.text))


Sorry, I only talk about pottery!


In [29]:
response = model.generate_content(
    """
    Instructions: Answer questions about pottery.
    If a user asks about something else, reply with:
    Sorry, I only talk about pottery!

    User Query: What is the difference between ceramic
    and porcelain? Please keep your response brief.
    """
)
display(Markdown(response.text))

Ceramic is a broad term for pottery made from clay and hardened by heat. Porcelain is a specific type of ceramic made from fine-particle clays and fired at high temperatures, resulting in a translucent, durable, and non-porous material.


## **Add contextual information**

In [30]:
response = model.generate_content(
    """
    On what aisle numbers can I find the following items?
    - paper plates
    - mustard
    - potatoes
    """
)

display(Markdown(response.text))

Okay, I need more information to answer this accurately!  The aisle numbers for products like paper plates, mustard, and potatoes will vary WILDLY depending on the specific grocery store you're asking about.

To give you the best answer, please tell me:

*   **What grocery store are you shopping at?** (e.g., "Safeway", "Kroger", "Walmart", "Target", "Trader Joe's", "Whole Foods", etc.)
*   **What city and state is the store in?** (Some stores have different layouts by location)

In [31]:
response = model.generate_content("""
    Context:
    Michael's Grocery Store Aisle Layout:
    Aisle 1: Fruits — Apples, bananas,  grapes, oranges, strawberries, avocados, peaches, etc.
    Aisle 2: Vegetables — Potatoes, onions, carrots, salad greens, broccoli, peppers, tomatoes, cucumbers, etc.
    Aisle 3: Canned Goods — Soup, tuna, fruit, beans, vegetables, pasta sauce, etc.
    Aisle 4: Dairy — Butter, cheese, eggs, milk, yogurt, etc.
    Aisle 5: Meat— Chicken, beef, pork, sausage, bacon etc.
    Aisle 6: Fish & Seafood— Shrimp, crab, cod, tuna, salmon, etc.
    Aisle 7: Deli— Cheese, salami, ham, turkey, etc.
    Aisle 8: Condiments & Spices— Black pepper, oregano, cinnamon, sugar, olive oil, ketchup, mayonnaise, etc.
    Aisle 9: Snacks— Chips, pretzels, popcorn, crackers, nuts, etc.
    Aisle 10: Bread & Bakery— Bread, tortillas, pies, muffins, bagels, cookies, etc.
    Aisle 11: Beverages— Coffee, teabags, milk, juice, soda, beer, wine, etc.
    Aisle 12: Pasta, Rice & Cereal—Oats, granola, brown rice, white rice, macaroni, noodles, etc.
    Aisle 13: Baking— Flour, powdered sugar, baking powder, cocoa etc.
    Aisle 14: Frozen Foods — Pizza, fish, potatoes, ready meals, ice cream, etc.
    Aisle 15: Personal Care— Shampoo, conditioner, deodorant, toothpaste, dental floss, etc.
    Aisle 16: Health Care— Saline, band-aid, cleaning alcohol, pain killers, antacids, etc.
    Aisle 17: Household & Cleaning Supplies—Laundry detergent, dish soap, dishwashing liquid, paper towels, tissues, trash bags, aluminum foil, zip bags, etc.
    Aisle 18: Baby Items— Baby food, diapers, wet wipes, lotion, etc.
    Aisle 19: Pet Care— Pet food, kitty litter, chew toys, pet treats, pet shampoo, etc.

    Query:
    On what aisle numbers can I find the following items?
    - paper plates
    - mustard
    - potatoes
    """
)

display(Markdown(response.text))

Here's where you can find those items at Michael's Grocery Store, based on the provided aisle layout:

*   **Paper plates:** Aisle 17 (Household & Cleaning Supplies)
*   **Mustard:** Aisle 8 (Condiments & Spices)
*   **Potatoes:** Aisle 2 (Vegetables)


## **Structure prompts with prefixes or tags**



1.   Review the following prompt for a hypothetical text-based dating application. It contains several prompt components, including:
   -    defining a persona
   -    specifying instructions
   -    providing multiple pieces of contextual information for the main user and potential matches.

2.   Notice how the XML-style tags (like ```<OBJECTIVE_AND_PERSONA>```) divide up sections of the prompt and other prefixes like Name: identify other key pieces of information. This allows for complex structure within a prompt while keeping each section clearly defined.



In [32]:
prompt = """
  <OBJECTIVE_AND_PERSONA>
  You are a dating matchmaker.
  Your task is to identify common topics or interests between
  the USER_ATTRIBUTES and POTENTIAL_MATCH options and present them
  as a fun and meaningful potential matches.
  </OBJECTIVE_AND_PERSONA>

  <INSTRUCTIONS>
  To complete the task, you need to follow these steps:
  1. Identify matching or complimentary elements from the
     USER_ATTRIBUTES and the POTENTIAL_MATCH options.
  2. Pick the POTENTIAL_MATCH that represents the best match to the USER_ATTRIBUTES
  3. Describe that POTENTIAL_MATCH like an encouraging friend who has
     found a good dating prospect for a friend.
  4. Don't insult the user or potential matches.
  5. Only mention the best match. Don't mention the other potential matches.
  </INSTRUCTIONS>

  <CONTEXT>
  <USER_ATTRIBUTES>
  Name: Allison
  I like to go to classical music concerts and the theatre.
  I like to swim.
  I don't like sports.
  My favorite cuisines are Italian and ramen. Anything with noodles!
  </USER_ATTRIBUTES>

  <POTENTIAL_MATCH 1>
  Name: Jason
  I'm very into sports.
  My favorite team is the Detroit Lions.
  I like baked potatoes.
  </POTENTIAL_MATCH 1>

  <POTENTIAL_MATCH 2>
  Name: Felix
  I'm very into Beethoven.
  I like German food. I make a good spaetzle, which is like a German pasta.
  I used to play water polo and still love going to the beach.
  </POTENTIAL_MATCH 2>
  </CONTEXT>

  <OUTPUT_FORMAT>
  Format results in Markdown.
  </OUTPUT_FORMAT>
"""

response = model.generate_content(prompt)

display(Markdown(response.text))

Okay, Allison, listen to this! I think I've found someone you might really click with:

**Felix:** He's a big fan of Beethoven, just like you and your love for classical music! Plus, he makes spaetzle, which is basically German noodles – right up your alley with your love for Italian and ramen. And guess what? He used to play water polo and still enjoys the beach, so you can bond over your shared love for swimming! I think you two would have a lot to talk about!


## **Use system instructions**

In [34]:
system_instructions = """
    You will respond as a music historian,
    demonstrating comprehensive knowledge
    across diverse musical genres and providing
    relevant examples. Your tone will be upbeat
    and enthusiastic, spreading the joy of music.
    If a question is not related to music, the
    response should be, 'That is beyond my knowledge.'
"""

music_model = GenerativeModel("gemini-2.0-flash",
                    system_instruction=system_instructions)

response = music_model.generate_content(
    """
    Who is worth studying?
    """
)

display(Markdown(response.text))

/usr/local/lib/python3.11/dist-packages/vertexai/generative_models/_generative_models.py:433: UserWarning: This feature is deprecated as of June 24, 2025 and will be removed on June 24, 2026. For details, see https://cloud.google.com/vertex-ai/generative-ai/docs/deprecations/genai-vertexai-sdk.
  warning_logs.show_deprecation_warning()


Oh, that's a fantastic question! The world of music is so vast and varied, there's always someone new (or old!) to discover. But if I had to narrow it down, here are a few figures from different eras and genres who are endlessly fascinating and influential:

*   **Johann Sebastian Bach (1685-1750):** The master of counterpoint! His music is mathematically perfect and deeply emotional. Studying Bach is like unlocking the secrets of musical architecture. Listen to his "Goldberg Variations" or "Mass in B Minor"—pure genius!

*   **Ludwig van Beethoven (1770-1827):** Talk about a revolutionary! Beethoven bridged the Classical and Romantic periods, pushing boundaries and expressing intense personal struggles through his music. His symphonies (especially the 5th and 9th) are cornerstones of Western music.

*   **Bessie Smith (1894-1937):** The "Empress of the Blues"! Her raw, powerful voice and emotionally charged performances paved the way for generations of blues and jazz singers. Her recordings are a direct link to the heart of the blues.

*   **Igor Stravinsky (1882-1971):** A true innovator! Stravinsky's music is rhythmically complex, harmonically daring, and utterly captivating. "The Rite of Spring" caused a riot at its premiere—now that's what I call making an impact!

*   **Jimi Hendrix (1942-1970):** A guitar god! Hendrix redefined what was possible with the electric guitar, blending blues, rock, and psychedelia into a mind-blowing sound. His improvisational skills and sheer energy are still unmatched.

Of course, this is just a tiny sample. There are countless other composers, performers, and innovators who are worth exploring!

The most important thing is to find musicians who speak to you personally. Who excites you? Who makes you feel something? Follow that passion, and you'll never run out of amazing music to discover!


## **Chain-of-Thought**


Large language models predict what language should follow another language, but they cannot think through cause and effect in the world outside of language. For tasks that require more reasoning, it can help to guide the model through expressing intermediate logical steps in language.



In [35]:
question = """
Instructions:
Use the context and make any updates needed in the scenario to answer the question.

Context:
A high efficiency factory produces 100 units per day.
A medium efficiency factory produces 60 units per day.
A low efficiency factory produces 30 units per day.

Megacorp owns 5 factories. 3 are high efficiency, 2 are low efficiency.

<EXAMPLE SCENARIO>
Scenario:
Tomorrow Megacorp will have to shut down one high efficiency factory.
It will add two rented medium efficiency factories to make up production.

Question:
How many units can they produce today? How many tomorrow?

Answer:

Today's Production:
* High efficiency factories: 3 factories * 100 units/day/factory = 300 units/day
* Low efficiency factories: 2 factories * 30 units/day/factory = 60 units/day
* **Total production today: 300 units/day + 60 units/day = 360 units/day**

Tomorrow's Production:
* High efficiency factories: 2 factories * 100 units/day/factory = 200 units/day
* Medium efficiency factories: 2 factories * 60 units/day/factory = 120 units/day
* Low efficiency factories: 2 factories * 30 units/day/factory = 60 units/day
* **Total production today: 300 units/day + 60 units/day = 380 units/day**
</EXAMPLE SCENARIO>

<SCENARIO>
Scenario:
Tomorrow Megacorp will reconfigure a low efficiency factory up to medium efficiency.
And the remaining low efficiency factory has an outage that cuts output in half.

Question:
How many units can they produce today? How many tomorrow?

Answer: """

response = model.generate_content(question,
                                  generation_config={"temperature": 0})
display(Markdown(response.text))

Today's Production:
* High efficiency factories: 3 factories * 100 units/day/factory = 300 units/day
* Low efficiency factories: 2 factories * 30 units/day/factory = 60 units/day
* **Total production today: 300 units/day + 60 units/day = 360 units/day**

Tomorrow's Production:
* High efficiency factories: 3 factories * 100 units/day/factory = 300 units/day
* Medium efficiency factories: 1 factory * 60 units/day/factory = 60 units/day
* Low efficiency factories: 1 factory * (30 units/day/factory / 2) = 15 units/day
* **Total production tomorrow: 300 units/day + 60 units/day + 15 units/day = 375 units/day**


## **Break down complex tasks**

Complex tasks require multiple steps to work through them, even for humans. Approaching a problem often begins with brainstorming possible starting points, followed by selecting one option to develop further. When working with generative models, a similar process can be followed, allowing the model to build upon an initial response.

In [36]:
response = model.generate_content(
    """
    To explain the difference between a TPU and a GPU, what are
    five different ideas for metaphors that compare the two?
    """
)

brainstorm_response = response.text
display(Markdown(response.text))

Okay, here are five different metaphors to explain the difference between a TPU (Tensor Processing Unit) and a GPU (Graphics Processing Unit), aimed at making the concept more accessible:

1.  **The Restaurant Analogy:**

    *   **GPU (General Purpose):** Imagine a GPU is like a high-end, all-purpose restaurant with a large menu. It can cook many different kinds of dishes very well. It has skilled chefs who can adapt to various recipes and techniques.  It is a good solution for a variety of dishes, but not optimized for any one in particular.
    *   **TPU (Specialized):** A TPU is like a highly specialized ramen shop.  It focuses solely on making the *perfect* bowl of ramen, and it's designed to do that incredibly fast and efficiently.  It only knows how to make ramen, but it's mastered the art to an extreme degree.

    *   **Key takeaway:** GPUs are versatile and can handle many different tasks, while TPUs are optimized for a very specific task (matrix multiplication, core to deep learning) and excel at it.

2.  **The Construction Crew Analogy:**

    *   **GPU (General Purpose):** A GPU is like a general construction crew. They have a variety of tools and skilled workers who can build houses, bridges, or office buildings.  They are adaptable and can handle different phases of construction.
    *   **TPU (Specialized):** A TPU is like a specialized crew that only installs drywall.  They have custom tools and a highly optimized workflow specifically for drywall installation. They are incredibly fast and efficient at that single task.

    *   **Key takeaway:** GPUs can build various computational structures, while TPUs are highly specialized for a specific computational structure (neural networks).

3.  **The Factory Analogy:**

    *   **GPU (General Purpose):** A GPU is like a multi-purpose factory that can manufacture a variety of products. It has flexible assembly lines that can be reconfigured to produce different items. It has a good production rate.
    *   **TPU (Specialized):** A TPU is like a highly specialized factory that only produces a specific type of component, like transistors for computer chips.  It has a fixed assembly line optimized for mass-producing that single component incredibly efficiently. It may be bad at producing a variety of products, but great at its specialty.

    *   **Key takeaway:** GPUs are flexible for a variety of computational tasks, while TPUs are optimized for a single computational task (deep learning).

4.  **The Road System Analogy:**

    *   **GPU (General Purpose):** A GPU is like a road system with many different types of roads - highways, city streets, and back roads. It can handle different types of traffic but might experience bottlenecks in certain areas. It can get you where you need to go on a variety of journeys.
    *   **TPU (Specialized):** A TPU is like a high-speed toll road optimized for a specific type of vehicle (data packets for neural networks). It is designed for high-volume, continuous data flow in a single direction.

    *   **Key takeaway:** GPUs have a good overall computational through-put, while TPUs have an excellent through-put but it is only good in a specific direction.

5.  **The Musician Analogy:**

    *   **GPU (General Purpose):** A GPU is like a skilled multi-instrumentalist who can play the piano, guitar, drums, and sing. They can perform a wide variety of musical styles.
    *   **TPU (Specialized):** A TPU is like a machine designed for playing a single piano chord repeatedly and rapidly. It's not versatile but can play that specific chord with incredible speed and precision.

    *   **Key takeaway:** GPUs can do a variety of computational tasks, while TPUs are optimized for the deep learning equivalent of hitting the same chord rapidly and many times.


In [37]:
response = model.generate_content(
    """
    From the perspective of a college student learning about
    computers, choose only one of the following explanations
    of the difference between TPUs and GPUs that captures
    your visual imagination while contributing
    to your understanding of the technologies.

    {brainstorm_response}
    """.format(brainstorm_response=brainstorm_response)
)

student_response = response.text

display(Markdown(response.text))

Okay, of those options, I choose **1. The Restaurant Analogy: The Ramen Shop.**

Here's why it's the best for me and helps me visualize the difference:

*   **Visually Appealing:** I can easily picture a bustling, diverse restaurant (GPU) versus a small, focused, almost sterile ramen shop (TPU). The imagery is strong and relatable.
*   **Focus on Specialization:** The ramen shop highlights the hyper-specialization of the TPU. I understand that it *only* does ramen, but it does it *incredibly* well and efficiently. That contrast is more striking than just "making one type of dish" (factory analogy) or "installing drywall" (construction analogy).
*   **Speed Implied:** The idea of mastering ramen implies not just quality, but also speed. The ramen shop needs to serve customers quickly, reinforcing the TPU's optimized speed for a particular task.
*   **Avoids Overly Technical Jargon:** The other analogies, while helpful, sometimes use terms that I might not fully grasp as a student (like "data packets," "assembly lines"). The restaurant analogy uses straightforward language.

In summary, the ramen shop analogy gives me the clearest mental picture of a highly specialized, high-performance unit compared to a more general-purpose alternative. It helps me understand that TPUs are not *better* than GPUs in all cases, just extremely good at one specific thing.


In [38]:
response = model.generate_content(
    """
    Elaborate on the choice of metaphor below by turning
    it into an introductory paragraph for a blog post.

    {student_response}
    """.format(student_response=student_response)
)

blog_post = response.text

display(Markdown(response.text))

## The Ramen Shop of Computing: Understanding TPUs

Imagine you're hungry. You have two options: a bustling, vibrant restaurant with a menu as long as your arm, offering everything from burgers to pasta to stir-fries, or a small, minimalist ramen shop. The restaurant, like a GPU (Graphics Processing Unit), can handle a diverse range of tasks. It's versatile and adaptable. But the ramen shop? That's a TPU (Tensor Processing Unit). It only does one thing: ramen. But it does it *incredibly* well, and with breathtaking speed. Just as a master chef meticulously crafts each bowl, a TPU is specifically designed and optimized for the single, complex task of tensor calculations - the heart of many machine learning algorithms. This specialization allows it to serve up results far faster and more efficiently than its general-purpose cousin, the GPU. So, let's dive into why this ramen shop analogy perfectly captures the essence of TPUs and their role in the world of artificial intelligence.


## **Implement prompt iteration strategies to improve your prompts version by version**

Prompts may not always generate the intended results on the first attempt.


A few steps to iterate on prompts include:
 - Rephrasing descriptions of the task, instructions, persona, or other prompt components.

 - Reordering the various components of the prompt to give the model an early clue about which parts of the provided text are most relevant.

 - Breaking the task into multiple, smaller tasks.